## set running directory to project source

In [1]:
import os
import numpy as np
import time
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))
import matplotlib.pyplot as plt

## 4.1 PlanningScene

##### initialize CombinedRobot and GeometryScene

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *
from pkg.geometry.builder.scene_builder import SceneBuilder

s_builder = SceneBuilder(None)   # create scene builder without detector for virtual scene
crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7gripper, ((0,0,0.01), (0,0,0)), None)]
                     , connection_list=[False])
gscene = s_builder.create_gscene(crob)
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
gscene.show_pose(crob.home_pose)

connection command:
indy0: False
Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


##### add geometries

In [3]:
from pkg.geometry.geometry import *

# add environments (fixed=True for non-movable geometries)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.2,0,0), 
                           rpy=(0,np.pi/2,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (3,3,0.01), (0,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
wp1 = gscene.create_safe(GEOTYPE.BOX, "wp1", "base_link", (0.1,0.1,0.01), (0.6,-0.3,0),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=True, parent="floor")
wp2 = gscene.create_safe(GEOTYPE.BOX, "wp2", "base_link", (0.1,0.1,0.01), (0.2,-0.3,0), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=True, parent="floor")
goal = gscene.create_safe(GEOTYPE.BOX, "goal", "base_link", (0.1,0.1,0.01), (0.3,-0.4,0), 
                          rpy=(0,0,0), color=(0.2,0.2,0.8,1), display=True, fixed=True, collision=True)
 
# add movable (fixed=False for movable geometries)
box1 = gscene.create_safe(GEOTYPE.BOX, "box1", "base_link", (0.05,0.05,0.05), (0.3,0.4,0.031), 
                          rpy=(0,0,0), color=(0.8,0.2,0.2,1), display=True, fixed=False, collision=True)

obstacle = gscene.create_safe(GEOTYPE.BOX, "obstacle", "base_link", (0.05,0.05,0.05), (0.5,0.4,0.031), 
                          rpy=(0,0,0), color=(0.8,0.8,0.2,1), display=True, fixed=False, collision=True)

##### create PlanningScene

In [4]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

##### create_binder
- Binders (or Actors) are binding points where objects can be attached (or binded)
- Examples are 
  - PlacePlane: plane for object placement
  - Gripper2Tool: 2-finger gripper tool for grasp objects
  - SweepTool: action point to pass waypoints for sweep task

In [5]:
from pkg.planning.constraint.constraint_actor import PlacePlane, Gripper2Tool, SweepFramer

In [6]:
# create PlacePlane on geometry "floor" and "goal"
# when point is not set, the entire upper surface of the geometry becomes valid binding area.
# when point is set, the specific point becomes the only valid binding point.
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane)
pscene.create_binder(bname="goal", gname="goal", _type=PlacePlane, point=(0,0,0.005))

In [7]:
# add collision boundary for gripper base
# - set link_name="indy0_tcp" to attach the geometry to end-effector link
# - it can be labeled as fixed=True, as it is "fixed" on the indy0_tcp link
gripper =  gscene.create_safe(GEOTYPE.SPHERE, "grip0", link_name="indy0_tcp", 
                                dims=(0.01,0.01,0.01), center=(0,0,0.14), rpy=(-np.pi/2,0,0), 
                                color=(0.8,0.2,0.2,0.5), display=False, fixed=True, collision=False)

# create Gripper2Tool binder
# Gripper2Tool is a 2-finger gripper, which can rotate along z-direction.
# To align the z-direction with the 2 fingers, rotate by 90 degree along roll axis.
# The gripping point is (0,0,0.11) in local coordinate of "gripper" geometry
pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))

In [8]:
# Add virtual (no-collision) sweep face. the point is 0.2 m away from the "indy0_tcp" link
# To match the z-direction with the target surface, the geometry is rotated 180 degrees in pitch-axis.
sweep_face =  gscene.create_safe(GEOTYPE.BOX, "sweep_face", link_name="indy0_tcp", 
                                dims=(0.05,0.05,0.001), center=(0,0,0.2), rpy=(0,np.pi,0), 
                                color=(0.2,0.2,0.8,0.1), display=False, fixed=True, collision=False)
                                 
# create SweepTool binder
pscene.create_binder(bname="sweep_face", gname="sweep_face", _type=SweepFramer, point=(0,0,0), rpy=(0,0,0))

##### create_subject
* Subject describes the tasks in the planning scene.
* There are 2 categories in subject:
  1. Object: The object has grip points and placement points for pick&place task
  2. Task: The task is can be any non-physical task. Check SweepLineTask for example
* The subjects can be composed of multiple action points. Examples are:
  1. Grasp2Point: grasping point for 2-finger gripper. 
  2. PlacePoint: The point to place object.
  3. SweepPoint: A waypoint for SweepLineTask.
  * The above 3 action points inherit DirectePoint, for which the orientation is free along z-axis. 
  * If "point" parameter is not set, the entire upper surface is becomes valid action area.

In [9]:
from pkg.planning.constraint.constraint_subject import Grasp2Point, PlacePoint, SweepFrame
from pkg.planning.constraint.constraint_subject import CustomObject, SweepLineTask

In [10]:
## create box object with grasping points along positive & negative y-direction and placement point in the bottom face
box_obj = pscene.create_subject(oname="box1", gname="box1", _type=CustomObject, 
                             action_points_dict = {
                                 "handle1": Grasp2Point("handle1", box1, [0,0,0], [-np.pi/2,0,0]),
                                 "handle2": Grasp2Point("handle2", box1, [0,0,0], [np.pi/2,0,0]),
                                 "bottom": PlacePoint("bottom", box1, [0,0,-0.026], [0,0,0])})

In [11]:
## create box object with grasping points along positive & negative y-direction and placement point in the bottom face
obs_obj = pscene.create_subject(oname="obstacle", gname="obstacle", _type=CustomObject, 
                             action_points_dict = {
                                 "handle1": Grasp2Point("handle1", obstacle, [0,0,0], [-np.pi/2,0,0]),
                                 "handle2": Grasp2Point("handle2", obstacle, [0,0,0], [np.pi/2,0,0]),
                                 "bottom": PlacePoint("bottom", obstacle, [0,0,-0.026], [0,0,0])})

In [12]:
## create sweep task with 2 waypoints
sweep = pscene.create_subject(oname="sweep", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp1": SweepFrame("wp1", wp1, [0,0,0.005], [0,0,0]),
                                                   "wp2": SweepFrame("wp2", wp2, [0,0,0.005], [0,0,0])}
                             )

## Pipeline

In [13]:
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

In [14]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


##### initialize_state
* initialize_state(robot_pose) updates robot pose and gets corresponding binding status of current scene.
* state.node of ('floor', 0) means the first subject (object) is placed on the floor and the second subject (sweep) has passed 0 waypoints

In [15]:
initial_state = pscene.initialize_state(crob.home_pose)
print(initial_state.node)

   Use a production WSGI server instead.
('floor', 'floor', 0) * Debug mode: off



## import planners

In [16]:

from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker
from pkg.planning.filtering.task_clearance_filter import TaskClearanceChecker

gcheck = GraspChecker(pscene)
rcheck = ReachChecker(pscene)
checkers_all = [rcheck, gcheck]
# tcheck = TaskClearanceChecker(pscene, gcheck)
# checkers_all = [tcheck, rcheck, gcheck]
# lcheck = LatticedChecker(pscene, gcheck)
# checkers_all.append(lcheck)

## Test pybullet

In [17]:
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src/scripts/developing/pddlstream'))

In [18]:
from convert_pscene import *
from plan_rnb import *

#### load pybullet, convert scene

In [19]:
connect_notebook(use_gui=False)
urdf_pybullet_path = copy_meshes(gscene)
robot_body, body_names, movable_bodies = pscene_to_pybullet(pscene, urdf_pybullet_path, tool_name="grip0", name_exclude_list=["indy0"])
print('Objects:', body_names)
saver = WorldSaver()

[Pybullet] Load urdf from /home/rnb/Projects/rnb-planning/src/robots/custom_robots_pybullet.urdf
[WARING] non-object subject not implemented for now
('Objects:', {1L: 'goal', 2L: 'floor', 3L: 'wall', 4L: 'obstacle', 5L: 'box1'})


### Test with IK checkers

In [20]:
gtimer = GlobalTimer.instance()
gtimer.reset()

In [21]:
log_wFeas = []
for _ in range(3):
    problem = pddlstream_from_problem_rnb(pscene, robot_body, movable=movable_bodies, checkers=checkers_all,
                                          tool_name="grip0", tool_link_name="indy0_tcp", mplan=mplan)
    _, _, _, stream_map, init, goal = problem
    print('Init:', init)
    print('Goal:', goal)
    print('Streams:', str_from_object(set(stream_map)))
    with Profiler():
        with LockRenderer(lock=not True):
            with gtimer.block("solve"):
                solution = solve(problem, algorithm='adaptive', unit_costs=False, success_cost=INF)
            saver.restore()
    print_solution(solution)
    plan, cost, evaluations = solution
    log_wFeas.append(cost)

Robot: 0
Movable: [4L, 5L]
Fixed: [1, 2, 3]
body 4 - surface 1
body 4 - surface 2
body 4 - surface 3
body 5 - surface 1
body 5 - surface 2
body 5 - surface 3
('Init:', [('CanMove',), ('Conf', q0), ('AtConf', q0), ('HandEmpty',), ('Graspable', 4L), ('Pose', 4L, p0), ('AtPose', 4L, p0), ('Stackable', 4L, 1), ('Stackable', 4L, 2), ('Supported', 4L, p0, 2), ('Stackable', 4L, 3), ('Graspable', 5L), ('Pose', 5L, p1), ('AtPose', 5L, p1), ('Stackable', 5L, 1), ('Stackable', 5L, 2), ('Supported', 5L, p1, 2), ('Stackable', 5L, 3)])
('Goal:', ('and', ('AtConf', q0), ('On', 4L, 1)))
('Streams:', '{TrajCollision, inverse-kinematics, plan-free-motion, plan-holding-motion, sample-grasp, sample-pose, test-cfree-approach-pose, test-cfree-pose-pose, test-cfree-traj-pose}')
Warning! All actions have no cost. Recommend setting unit_costs=True
Warning! Undeclared predicates: ['conf', 'graspable']
Setting negate=True for stream [test-cfree-approach-pose]
Setting negate=True for stream [test-cfree-pose-pose]

Attempt: 1 | Results: 24 | Depth: 1 | Success: False | Time: 0.082
Attempt: 2 | Results: 61 | Depth: 0 | Success: True | Time: 0.194
Stream plan (13, 13, 0.001): [sample-grasp:(4)->(#g1), inverse-kinematics:(4, p4, #g1)->(#q2, #t81), test-cfree-traj-pose:(#t81, 5, p5)->(), sample-pose:(4, 1)->(#p1), inverse-kinematics:(4, #p1, #g1)->(#q3, #t82), test-cfree-traj-pose:(#t82, 5, p5)->(), test-cfree-traj-pose:(#t81, 4, p4)->(), test-cfree-pose-pose:(4, #p1, 5, p5)->(), test-cfree-approach-pose:(4, p4, #g1, 5, p5)->(), test-cfree-approach-pose:(4, #p1, #g1, 5, p5)->(), plan-free-motion:(#q3, q3)->(#t123), plan-free-motion:(q3, #q2)->(#t124), plan-holding-motion:(#q2, #q3, 4, #g1)->(#t125)]
Action plan (5, 0.000): [move_free(q3, #q2, #t124), pick(4, p4, #g1, #q2, #t81), move_holding(#q2, #q3, 4, #g1, #t125), place(4, #p1, #g1, #q3, #t82), move_free(#q3, q3, #t123)]
iter=0, outs=1) sample-grasp:(4)->[(g2)]
iter=inf, outs=1) inverse-kinematics:(4, p4, g2)->[(q4, c5)]
iter=inf, outs=1) test-cfr

         372622 function calls (359415 primitive calls) in 0.527 seconds

   Ordered by: internal time
   List reduced from 1534 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.117    0.039    0.117    0.039 pkg/planning/motion/moveit/moveit_py.py:156(plan_joint_motion_py)
      528    0.033    0.000    0.033    0.000 {pybullet.calculateInverseKinematics}
        2    0.029    0.015    0.029    0.015 {posix.read}
        2    0.016    0.008    0.016    0.008 {posix.fork}
     3636    0.013    0.000    0.013    0.000 {pybullet.resetJointState}
      456    0.011    0.000    0.011    0.000 {method 'read' of 'file' objects}
  3886/38    0.009    0.000    0.026    0.001 /usr/lib/python2.7/copy.py:145(deepcopy)
     5232    0.005    0.000    0.006    0.000 /home/rnb/Projects/pddlstream/pddlstream/algorithms/../../FastDownward/builds/release64/bin/translate/pddl/conditions.py:230(__init__)
    26483    0.004    0.000    0.

In [22]:
print(gtimer)

solve: 	1790.0 ms/3 = 596.7 ms (529.71/721.993)
ik_fn: 	555.2 ms/13 = 42.7 ms (1.367/99.13)
test: 	22.5 ms/13 = 1.7 ms (1.284/2.682)
ReachChecker: 	7.2 ms/13 = 0.6 ms (0.247/1.213)
GraspChecker: 	8.2 ms/11 = 0.7 ms (0.556/0.919)
free_motion_gen: 	286.7 ms/6 = 47.8 ms (36.361/52.886)
holding_motion_gen: 	130.8 ms/3 = 43.6 ms (36.646/52.146)



In [23]:
print(log_wFeas)

[0.0, 0.0, 0.0]


### Test without IK checkers

In [24]:
gtimer = GlobalTimer.instance()
gtimer.reset()

In [25]:
log_woFeas = []
for _ in range(3):
    problem = pddlstream_from_problem_rnb(pscene, robot_body, movable=movable_bodies, checkers=[],
                                          tool_name="grip0", tool_link_name="indy0_tcp", mplan=mplan)
    _, _, _, stream_map, init, goal = problem
    print('Init:', init)
    print('Goal:', goal)
    print('Streams:', str_from_object(set(stream_map)))
    with Profiler():
        with LockRenderer(lock=not True):
            with gtimer.block("solve"):
                solution = solve(problem, algorithm='adaptive', unit_costs=False, success_cost=INF)
            saver.restore()
    print_solution(solution)
    plan, cost, evaluations = solution
    log_woFeas.append(cost)

Robot: 0
Movable: [4L, 5L]
Fixed: [1, 2, 3]
body 4 - surface 1
body 4 - surface 2
body 4 - surface 3
body 5 - surface 1
body 5 - surface 2
body 5 - surface 3
('Init:', [('CanMove',), ('Conf', q9), ('AtConf', q9), ('HandEmpty',), ('Graspable', 4L), ('Pose', 4L, p11), ('AtPose', 4L, p11), ('Stackable', 4L, 1), ('Stackable', 4L, 2), ('Supported', 4L, p11, 2), ('Stackable', 4L, 3), ('Graspable', 5L), ('Pose', 5L, p12), ('AtPose', 5L, p12), ('Stackable', 5L, 1), ('Stackable', 5L, 2), ('Supported', 5L, p12, 2), ('Stackable', 5L, 3)])
('Goal:', ('and', ('AtConf', q9), ('On', 4L, 1)))
('Streams:', '{TrajCollision, inverse-kinematics, plan-free-motion, plan-holding-motion, sample-grasp, sample-pose, test-cfree-approach-pose, test-cfree-pose-pose, test-cfree-traj-pose}')
Warning! All actions have no cost. Recommend setting unit_costs=True
Warning! Undeclared predicates: ['conf', 'graspable']
Setting negate=True for stream [test-cfree-traj-pose]
Setting negate=True for stream [test-cfree-pose-pos

         2412642 function calls (2366515 primitive calls) in 2.851 seconds

   Ordered by: internal time
   List reduced from 1508 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    12956    0.801    0.000    0.801    0.000 {pybullet.calculateInverseKinematics}
    78756    0.264    0.000    0.264    0.000 {pybullet.resetJointState}
        3    0.107    0.036    0.107    0.036 pkg/planning/motion/moveit/moveit_py.py:156(plan_joint_motion_py)
    13028    0.086    0.000    0.227    0.000 /usr/local/lib/python2.7/dist-packages/numpy/core/numeric.py:2508(within_tol)
        5    0.071    0.014    0.071    0.014 {posix.read}
    39697    0.059    0.000    0.059    0.000 {method 'reduce' of 'numpy.ufunc' objects}
    26056    0.051    0.000    0.102    0.000 /usr/local/lib/python2.7/dist-packages/numpy/core/numeric.py:2656(seterr)
    13028    0.046    0.000    0.422    0.000 /usr/local/lib/python2.7/dist-packages/numpy/core/numeric.p

Attempt: 2 | Results: 61 | Depth: 0 | Success: True | Time: 0.200
Stream plan (13, 13, 0.001): [sample-grasp:(4)->(#g5), inverse-kinematics:(4, p22, #g5)->(#q16, #t679), test-cfree-traj-pose:(#t679, 5, p23)->(), sample-pose:(4, 1)->(#p7), inverse-kinematics:(4, #p7, #g5)->(#q17, #t680), test-cfree-traj-pose:(#t680, 5, p23)->(), test-cfree-traj-pose:(#t679, 4, p22)->(), test-cfree-pose-pose:(4, #p7, 5, p23)->(), test-cfree-approach-pose:(4, p22, #g5, 5, p23)->(), test-cfree-approach-pose:(4, #p7, #g5, 5, p23)->(), plan-free-motion:(#q17, q21)->(#t721), plan-free-motion:(q21, #q16)->(#t723), plan-holding-motion:(#q16, #q17, 4, #g5)->(#t722)]
Action plan (5, 0.000): [move_free(q21, #q16, #t723), pick(4, p22, #g5, #q16, #t679), move_holding(#q16, #q17, 4, #g5, #t722), place(4, #p7, #g5, #q17, #t680), move_free(#q17, q21, #t721)]
iter=0, outs=1) sample-grasp:(4)->[(g13)]
iter=inf, outs=0) inverse-kinematics:(4, p22, g13)->[]
Sampling for up to 0.152 seconds
iter=1, outs=1) sample-grasp:(4)-

In [26]:
print(gtimer)

solve: 	4964.1 ms/3 = 1654.7 ms (851.773/2869.74)
ik_fn: 	2972.1 ms/26 = 114.3 ms (6.034/301.155)
test: 	14.8 ms/26 = 0.6 ms (0.264/1.265)
free_motion_gen: 	302.3 ms/6 = 50.4 ms (35.213/71.263)
holding_motion_gen: 	112.7 ms/3 = 37.6 ms (33.651/44.342)



In [27]:
print(log_wFeas)

[0.0, 0.0, 0.0]


## play plan

In [25]:
SIMULATE = False
if (plan is None) or not has_gui():
    disconnect()
else:
    command = postprocess_plan(plan)
    if SIMULATE:
#         wait_for_user('Simulate?')
        command.control()
    else:
#         wait_for_user('Execute?')
        #command.step()
        command.refine(num_steps=10).execute(time_step=0.001)